In [1]:
import re
import collections

### 求解
$$argmaxc ~ P(c|w) = argmaxc ~  \frac{P(w|c)P(c)}{P(w)}$$

* $P(c)$, 文章中出现一个正确拼写词 c 的概率；即在英语文章中, c 出现的概率有多大
* $P(w|c)$, 在用户想键入 c 的情况下敲成 w 的概率；代表用户会以多大的概率把 c 敲错成 w
* $argmaxc$, 枚举所有可能的 c 并且选取概率最大的

## 1. 数据预处理

In [2]:
# 取出语料库中所有单词，转化为小写并去除中间的特殊符号
# 返回的是文章中所有单词的列表

def words(text):
    return re.findall('[a-z]+', text.lower())

In [3]:
# 词频默认为1，避免未出现单词的概率为0

def train(word_list):
    model = collections.defaultdict(lambda: 1)
    
    for word in word_list:
        model[word] += 1
    
    return model

In [4]:
NWORDS = train(words(open('data.txt').read()))

## 2. 编辑距离

In [5]:
#返回所有与单词 w 编辑距离为 1 的集合

def edits1(word):
    n = len(word)
    return set([word[0:i]+word[i+1:] for i in range(n)] +                     # deletion
               [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] + # transposition
               [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] + # alteration
               [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])  # insertion

In [6]:
#返回所有与单词 w 编辑距离为 2 的集合
#在这些编辑距离小于2的词中间, 只把那些正确的词作为候选词

def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

## 3. 模拟实现拼写纠错

In [7]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [8]:
def known(words):
    return set(w for w in words if w in NWORDS)

In [9]:
# 如果known(set)非空, candidate 就会选取这个集合, 而不继续计算后面的
# 通过编辑距离相当于找出可能出现的单词，再通过词表可以得到正确的单词，并取出现次数最大的作为输出结果

def correct(word):
    candidates = known([word]) or known(edits1(word)) or edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])

In [10]:
correct("hera")

'her'